# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### For this problem, we can apply Bootstrap Resampling Technique in order to deal with this problem.
### Principles of CLT are definately appicable in our case.

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

### Total Whites and Black Count 

In [6]:
data.shape

(4870, 65)

### Total number of candidates taken into account is 4870

In [7]:
w.shape

(2435, 65)

In [8]:
b.shape

(2435, 65)

##### Total number of Whites: 2435
##### Total number of Blacks: 2435   

In [9]:
whites_getting_calls = w[w.call==1]
black_getting_calls = b[b.call==1]

In [10]:
whites_getting_calls.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
85,b,1,2,3,7,0,1,0,1,365,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
95,b,1,2,3,4,0,1,0,1,365,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
105,b,1,4,2,6,1,1,0,0,266,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Public
127,b,1,3,2,5,0,1,0,1,267,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,
165,b,1,4,5,10,0,0,0,1,264,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,


In [11]:
black_getting_calls.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
107,b,1,4,3,6,0,1,0,1,263,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Public
126,b,1,4,1,9,1,1,0,0,266,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,
171,b,1,3,3,11,0,0,0,1,903,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,
200,b,10,4,4,8,0,0,0,0,316,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Private
259,b,10,4,4,3,0,0,0,1,27,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private


In [12]:
black_getting_calls.shape

(157, 65)

#### Number of Black candidates getting calls: 157

In [13]:
whites_getting_calls.shape

(235, 65)

#### Number of Black candidates getting calls: 235

##### Null Hypothesis: To answer this question, we will evaluate the hypothesis that the candidate's race has no bearing on his or her interview call. We will use the fraction of white candidates getting interview call as statistic and evaluate the probability of observing a fraction of white candidates getting interview call at least as small as the observed fraction of 235/2435.

In [14]:
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

In [15]:
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

In [16]:
# Construct arrays of data: dems, reps
whites = np.array([True] * 235 + [False] * 2200)
blacks = np.array([True] * 157 + [False] * 2278)

def whites_get_interview(dems, reps):
    """Compute fraction of Democrat yay votes."""
    frac = np.sum(whites) / len(whites)
    return frac

# Acquire permutation samples: perm_replicates
perm_replicates = draw_perm_reps(whites, blacks, whites_get_interview,10000)

# Compute and print p-value: p
p = np.sum(perm_replicates >= 157/2435) / len(perm_replicates)
print('p-value =', p)

p-value = 1.0


In [ ]:
import numpy as np

def diff_frac(data_A,data_B):
    frac_A = np.sum(data_A)/len(data_A)
    frac_B = np.sum(data_B)/len(data_B)
    return frac_B - frac_A

diff_frac_obs = diff_frac(white,black)
perm_replicates = np.empty(10000)

for i in range(10000):
    perm_replicates[i] = permutation_replicate(white,black,diff_frac)

p_value = np.sum(perm_replicates >= diff_frac_obs)/10000

print(p_value)
    

### This large p-value suggests that candidate's race had nothing to do with getting interview calls.

In [17]:
# Your solution to Q3 here